## **Neural Network Implementation from scratch using "pima indians-diabetes-database"**

In [ ]:
# Install Kaggle API
pip install kaggle

In [ ]:
# from google.colab import files

from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mdyalam","key":"c2e739ee73a646cad89e903c817a0649"}'}

In [ ]:
# Move kaggle.json to the Correct Directory
import os
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# List all dataset in Kaggle
!kaggle datasets list

ref                                                              title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
asinow/car-price-dataset                                         Car Price Dataset                                135KB  2025-01-26 19:53:28          20468        329  1.0              
abdulmalik1518/mobiles-dataset-2025                              Mobiles Dataset (2025)                            20KB  2025-02-18 06:50:24           4084         79  1.0              
mahmoudelhemaly/students-grading-dataset                         Student Performance & Behavior Dataset           508KB  2025-02-17 17:38:46           3587         69  1.0              
adilshamim8/workout-and-fitness-tracker-data                     Worko

In [ ]:
# Download the Pima Indians Diabetes Dataset
!kaggle datasets download -d uciml/pima-indians-diabetes-database


Dataset URL: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database
License(s): CC0-1.0
  0% 0.00/8.91k [00:00<?, ?B/s]
100% 8.91k/8.91k [00:00<00:00, 13.4MB/s]


In [ ]:
# Extract the Dataset
import zipfile

with zipfile.ZipFile("pima-indians-diabetes-database.zip", "r") as zip_ref:
    zip_ref.extractall("pima_dataset")

# List extracted files
os.listdir("pima_dataset")


['diabetes.csv']

In [ ]:
# Load the Dataset into Pandas
import pandas as pd

df = pd.read_csv("pima_dataset/diabetes.csv")
print(df.head())


   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Load Pima dataset
df = pd.read_csv("pima_dataset/diabetes.csv")
X = df.iloc[:, :-1].values  # Features
y = df.iloc[:, -1].values   # Target

# One-hot encode the labels (Only if more than 2 classes)
if len(np.unique(y)) > 2:
    encoder = OneHotEncoder(sparse_output=False)
    y = encoder.fit_transform(y.reshape(-1, 1))
else:
    y = y.reshape(-1, 1)  # Keep as is for binary classification

# Standardize the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Neural Network Parameters
input_size = X_train.shape[1]
hidden_size = 10  # 10 neurons in the hidden layer
output_size = y_train.shape[1]  # Output classes (1 for binary, more for multi-class)

# Initialize weights and biases
np.random.seed(42)
W1 = np.random.randn(input_size, hidden_size) * 0.01
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * 0.01
b2 = np.zeros((1, output_size))

# Activation functions
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def cross_entropy_loss(y_pred, y_true):
    m = y_true.shape[0]
    return -np.sum(y_true * np.log(y_pred + 1e-8)) / m

# Forward Pass
def forward(X):
    z1 = np.dot(X, W1) + b1
    a1 = relu(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = sigmoid(z2) if output_size == 1 else softmax(z2)
    return a1, a2

# Backward Pass
def backward(X, y, a1, a2):
    m = X.shape[0]
    dz2 = a2 - y
    dW2 = np.dot(a1.T, dz2) / m
    db2 = np.sum(dz2, axis=0, keepdims=True) / m
    dz1 = np.dot(dz2, W2.T) * relu_derivative(a1)
    dW1 = np.dot(X.T, dz1) / m
    db1 = np.sum(dz1, axis=0, keepdims=True) / m
    return dW1, db1, dW2, db2

# Training the model
def train(X_train, y_train, learning_rate=0.01, epochs=1000):
    global W1, b1, W2, b2
    for epoch in range(epochs):
        a1, a2 = forward(X_train)
        loss = cross_entropy_loss(a2, y_train)
        dW1, db1, dW2, db2 = backward(X_train, y_train, a1, a2)
        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2
        if epoch % 100 == 0:
            print(f"Epoch {epoch}/{epochs}, Loss: {loss:.4f}")

# Predict function
def predict(X):
    _, a2 = forward(X)
    return (a2 > 0.5).astype(int) if output_size == 1 else np.argmax(a2, axis=1)

# Train the model
train(X_train, y_train, learning_rate=0.1, epochs=1000)

# Evaluate the model
y_pred_train = predict(X_train)
y_pred_test = predict(X_test)

y_true_train = y_train if output_size == 1 else np.argmax(y_train, axis=1)
y_true_test = y_test if output_size == 1 else np.argmax(y_test, axis=1)

train_accuracy = np.mean(y_pred_train == y_true_train) * 100
test_accuracy = np.mean(y_pred_test == y_true_test) * 100

print(f"\nTrain Accuracy: {train_accuracy:.2f}%")
print(f"\nTest Accuracy: {test_accuracy:.2f}%")


Epoch 0/1000, Loss: 0.2405
Epoch 100/1000, Loss: 0.3536
Epoch 200/1000, Loss: 0.3474
Epoch 300/1000, Loss: 0.2596
Epoch 400/1000, Loss: 0.2415
Epoch 500/1000, Loss: 0.2383
Epoch 600/1000, Loss: 0.2379
Epoch 700/1000, Loss: 0.2380
Epoch 800/1000, Loss: 0.2377
Epoch 900/1000, Loss: 0.2367

Train Accuracy: 79.32%

Test Accuracy: 77.27%


## **Neural Network Implementation from Scratch using "Fashion-MNIST Dataset"**

In [ ]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Load Fashion-MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Flatten the 28x28 images into 784-dimensional vectors
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Convert target labels to NumPy array
y_train = np.array(y_train)
y_test = np.array(y_test)

# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False)
y_train_one_hot = encoder.fit_transform(y_train.reshape(-1, 1))
y_test_one_hot = encoder.transform(y_test.reshape(-1, 1))

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Neural Network Parameters
input_size = X_train_scaled.shape[1]  # 784 features (28x28 pixels)
hidden_size = 128  # 128 neurons in the hidden layer
output_size = y_train_one_hot.shape[1]  # 10 output classes (clothing categories)

# Initialize weights and biases
np.random.seed(42)

W1 = np.random.randn(input_size, hidden_size) * 0.01  # Weight for input to hidden layer
b1 = np.zeros((1, hidden_size))  # Bias for hidden layer

W2 = np.random.randn(hidden_size, output_size) * 0.01  # Weight for hidden to output layer
b2 = np.zeros((1, output_size))  # Bias for output layer

# Activation functions and their derivatives
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # For numerical stability
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def cross_entropy_loss(y_pred, y_true):
    m = y_true.shape[0]
    return -np.sum(y_true * np.log(y_pred + 1e-8)) / m  # Add small epsilon for numerical stability

# Forward Pass
def forward(X):
    z1 = np.dot(X, W1) + b1
    a1 = relu(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = softmax(z2)
    return a1, a2

# Backward Pass (Gradient Descent and Backpropagation)
def backward(X, y, a1, a2):
    m = X.shape[0]

    # Output layer error
    dz2 = a2 - y
    dW2 = np.dot(a1.T, dz2) / m
    db2 = np.sum(dz2, axis=0, keepdims=True) / m

    # Hidden layer error
    dz1 = np.dot(dz2, W2.T) * relu_derivative(a1)
    dW1 = np.dot(X.T, dz1) / m
    db1 = np.sum(dz1, axis=0, keepdims=True) / m

    return dW1, db1, dW2, db2

# Training the model using gradient descent
def train(X_train, y_train, learning_rate=0.1, epochs=1000):
    global W1, b1, W2, b2

    for epoch in range(epochs):
        # Forward pass
        a1, a2 = forward(X_train)

        # Compute the loss
        loss = cross_entropy_loss(a2, y_train)

        # Backward pass
        dW1, db1, dW2, db2 = backward(X_train, y_train, a1, a2)

        # Update the weights and biases using gradient descent
        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2

        if epoch % 100 == 0:
            print(f"Epoch {epoch}/{epochs}, Loss: {loss:.4f}")

# Evaluating the model
def predict(X):
    _, a2 = forward(X)
    return np.argmax(a2, axis=1)

# Train the network
train(X_train_scaled, y_train_one_hot, learning_rate=0.1, epochs=1000)

# Evaluate the model
y_pred_train = predict(X_train_scaled)
y_pred_test = predict(X_test_scaled)

y_true_train = np.argmax(y_train_one_hot, axis=1)
y_true_test = np.argmax(y_test_one_hot, axis=1)

train_accuracy = np.mean(y_pred_train == y_true_train) * 100
test_accuracy = np.mean(y_pred_test == y_true_test) * 100

print(f"Train Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 0/1000, Loss: 2.3031
Epoch 100/1000, Loss: 0.5475
Epoch 200/1000, Loss: 0.4459
Epoch 300/1000, Loss: 0.4067
Epoch 400/1000, Loss: 0.3835
Epoch 500/1000, Loss: 0.3666
Epoch 600/1000, Loss: 0.3532
Epoch 700/1000, Loss: 0.3420
Epoch 800/1000, Loss: 0.3323
Epoch 900/1000, Loss: 0.3237
Train Accuracy: 88.81%
Test Accuracy: 86.46%
